In [5]:
!pip install bert4keras

  Created wheel for bert4keras: filename=bert4keras-0.7.7-cp36-none-any.whl size=36804 sha256=781bf7f25efd7c5f08077e4f330370f00da29ab2673f03e6722dd8e6c81d6047
  Stored in directory: /root/.cache/pip/wheels/73/5f/2b/ee7932b73172503fdb429421c0b433f47a737c0d1c2a85fdd2
Successfully built bert4keras


In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 重计算，减轻GPU压力
!export RECOMPUTE=1

In [0]:
!/opt/bin/nvidia-smi

Tue May 19 16:51:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

增加对抗训练缓解seq2seq生成系统的不稳定性

In [ ]:
"""
对于CV任务来说，一般输入张量的shape是(b,h,w,c)，这时候我们需要固定模型的batch size（即b），然后给原始输入加上一个shape同样为(b,h,w,c)、全零初始化的Variable，比如就叫做Δx，那么我们可以直接求loss对x的梯度，然后根据梯度给Δx赋值，来实现对输入的干扰，完成干扰之后再执行常规的梯度下降。

对于NLP任务来说，原则上也要对Embedding层的输出进行同样的操作，Embedding层的输出shape为(b,n,d)，所以也要在Embedding层的输出加上一个shape为(b,n,d)的Variable，然后进行上述步骤。但这样一来，我们需要拆解、重构模型，对使用者不够友好。

不过，我们可以退而求其次。Embedding层的输出是直接取自于Embedding参数矩阵的，因此我们可以直接对Embedding参数矩阵进行扰动。这样得到的对抗样本的多样性会少一些（因为不同样本的同一个token共用了相同的扰动），但仍然能起到正则化的作用，而且这样实现起来容易得多。
"""


def adversarial_training(model, embedding_name, epsilon=1):
    """给模型添加对抗训练
    其中model是需要添加对抗训练的keras模型，embedding_name
    则是model里边Embedding层的名字。要在模型compile之后使用。
    """
    if model.train_function is None:  # 如果还没有训练函数
        model._make_train_function()  # 手动make
    old_train_function = model.train_function  # 备份旧的训练函数

    # 查找Embedding层
    for output in model.outputs:
        embedding_layer = search_layer(output, embedding_name)
        if embedding_layer is not None:
            break
    if embedding_layer is None:
        raise Exception('Embedding layer not found')

    # 求Embedding梯度
    embeddings = embedding_layer.embeddings  # Embedding矩阵
    gradients = K.gradients(model.total_loss, [embeddings])  # Embedding梯度
    gradients = K.zeros_like(embeddings) + gradients[0]  # 转为dense tensor

    # 封装为函数
    inputs = (
        model._feed_inputs + model._feed_targets + model._feed_sample_weights
    )  # 所有输入层
    embedding_gradients = K.function(
        inputs=inputs,
        outputs=[gradients],
        name='embedding_gradients',
    )  # 封装为函数

    def train_function(inputs):  # 重新定义训练函数
        grads = embedding_gradients(inputs)[0]  # Embedding梯度
        delta = epsilon * grads / (np.sqrt((grads**2).sum()) + 1e-8)  # 计算扰动
        K.set_value(embeddings, K.eval(embeddings) + delta)  # 注入扰动
        outputs = old_train_function(inputs)  # 梯度下降
        K.set_value(embeddings, K.eval(embeddings) - delta)  # 删除扰动
        return outputs

    model.train_function = train_function  # 覆盖原训练函数

In [10]:
from __future__ import print_function
import glob
import numpy as np
from bert4keras.backend import keras, K, search_layer
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from keras.models import Model
from keras.layers import Input
import pandas as pd


# 基本参数
maxlen = 256
batch_size = 16
epochs = 40

# bert配置
config_path = '../content/drive/My Drive/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../content/drive/My Drive/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../content/drive/My Drive/chinese_L-12_H-768_A-12/vocab.txt'


# 导入数据
data = pd.read_csv('../content/drive/My Drive/项目4数据集.csv')

def load_data(data):
    D = []
    for i in range(len(data)):
            question = data.loc[i,"question"]
            answer = data.loc[i,"answer"]
            D.append((str(answer), str(question)))
    return D

train_data = load_data(data)

# 加载并精简词表，建立分词器
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)

Using TensorFlow backend.


In [11]:
train_data[:10]

[('您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。', '补发网银盾'),
 ('您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。', '代发工资'),
 ('您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。', '对帐单查询打印'),
 ('请说出您需要办理的业务', '理财产品取号'),
 ('请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。', '密码修改'),
 ('您可以到智慧柜员机办理此项业务，如需到柜台办理，请刷身份证取号', '查询名下所有账户'),
 ('您可以到智慧柜员机办理此项业务，如需到柜台办理，请刷身份证取号', '交易流水明细证明'),
 ('电视墙上有实时汇率表哦，您可以看看，如果您需要购汇或换汇，请到自助柜员机办理，如果您要取现钞，请刷身份证取号到柜台办理。如您使用护照或其他证件，可以请大堂经理过来帮您哦',
  '个人结售汇'),
 ('人民币百元钞2万以内可直接在ATM机办理，2万以上请刷身份证取号到柜台办理哦。', '现金取款'),
 ('您要办理的是个人转账、挂失、销户、修改密码业务。请刷身份证，小龙人来帮您取个号吧。', '账户销户')]

In [12]:
# 参考苏建林博客https://spaces.ac.cn/archives/7259，随机替换一下Decoder的输入词，缓解seq2seq的Exposure Bias
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        idxs = list(range(len(self.data)))
        if random:
            np.random.shuffle(idxs)
        batch_token_ids, batch_segment_ids, batch_o_token_ids = [], [], []
        for i in idxs:
            answer, question = self.data[i]
            token_ids, segment_ids = tokenizer.encode(question,
                                                      answer,
                                                      max_length=maxlen)
            o_token_ids = token_ids
            if np.random.random() > 0.5:
                token_ids = [
                    t if s == 0 or (s == 1 and np.random.random() > 0.3) else np.random.choice(token_ids)
                    for t, s in zip(token_ids, segment_ids)
                ]
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_o_token_ids.append(o_token_ids)
            if len(batch_token_ids) == self.batch_size or i == idxs[-1]:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_o_token_ids = sequence_padding(batch_o_token_ids)
                yield [batch_token_ids, batch_segment_ids, batch_o_token_ids], None
                batch_token_ids, batch_segment_ids, batch_o_token_ids = [], [], []


model = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # 只保留keep_tokens中的字，精简原字表
)

model.summary()

o_in = Input(shape=(None, ))
train_model = Model(model.inputs + [o_in], model.outputs + [o_in])

# 交叉熵作为loss，并mask掉输入部分的预测
y_true = train_model.input[2][:, 1:]  # 目标tokens
y_mask = train_model.input[1][:, 1:]
y_pred = train_model.output[0][:, :-1]  # 预测tokens，预测与目标错开一位
cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred)
cross_entropy = K.sum(cross_entropy * y_mask) / K.sum(y_mask)

train_model.add_loss(cross_entropy)
train_model.compile(optimizer=Adam(1e-5))
model.load_weights('../content/drive/My Drive/best_model_answer.weights')
adversarial_training(train_model, 'Embedding-Token', 0.5)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     multiple             10432512    Input-Token[0][0]                
                                                                 MLM-Norm[0][0]                   
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]        

/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output MLM-Activation missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to MLM-Activation.
  'be expecting any data to be passed to {0}.'.format(name))
/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output input_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to input_1.
  'be expecting any data to be passed to {0}.'.format(name))
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
class AutoAnswer(AutoRegressiveDecoder):
    """seq2seq解码器
    """
    @AutoRegressiveDecoder.set_rtype('probas')
    def predict(self, inputs, output_ids, step):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, np.ones_like(output_ids)], 1)
        return model.predict([token_ids, segment_ids])[:, -1]

    def generate(self, text, topk=3):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, max_length=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk)  # 基于beam search
        return tokenizer.decode(output_ids)


autoanswer = AutoAnswer(start_id=None, end_id=tokenizer._token_end_id, maxlen=96)


def just_show():
    s1 = u'我需要怎样去完成资产证明这个任务 。'
    s2 = u'你好我想问问怎么在智慧柜员机上激活信用卡。'
    for s in [s1, s2]:
        print(u'生成标题:', autoanswer.generate(s))
    print()


class Evaluate(keras.callbacks.Callback):
    def __init__(self):
        self.lowest = 1e10

    def on_epoch_end(self, epoch, logs=None):
        # 保存最优
        if logs['loss'] <= self.lowest:
            self.lowest = logs['loss']
            model.save_weights('../content/drive/My Drive/best_model_answer.weights')
        # 演示效果
        just_show()

In [14]:
if __name__ == '__main__':

    evaluator = Evaluate()
    train_generator = data_generator(train_data, batch_size)

    train_model.fit_generator(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        callbacks=[evaluator]
    )

else:

    model.load_weights('../content/drive/My Drive/best_model_answer.weights')

Epoch 1/40
2033/2033 [==============================] - 1719s 846ms/step - loss: 4.1585
生成标题: 我需要完成资产证明的任务。
生成标题: 您想激活信用卡的方法：登录个人网银，通过投资理财智慧柜员机激活信用卡。

Epoch 2/40
2033/2033 [==============================] - 1704s 838ms/step - loss: 3.4348
生成标题: 我需要完成资产证明的任务。
生成标题: 您好，有什么可以帮到您的吗？

Epoch 3/40
2033/2033 [==============================] - 1699s 835ms/step - loss: 3.1961
生成标题: 我需要完成资产证明的任务。
生成标题: 您好，智慧柜员机上激活信用卡的方法：登录个人网银，通过信用卡信用卡管理信用卡管理菜单，选择需激活的信用卡，点击激活按钮即可激活信用卡。

Epoch 4/40
2033/2033 [==============================] - 1707s 840ms/step - loss: 3.0323
生成标题: 我需要完成资产证明的任务。
生成标题: 您好，您可以在智慧柜员机上激活信用卡。

Epoch 5/40
2033/2033 [==============================] - 1698s 835ms/step - loss: 2.9010
生成标题: 您需要完成以下任务：<br/> 1、登录个人网银，通过我的账户账户管理账户管理账户管理菜单，选择需要完成的账户信息，点击确认按钮，根据页面提示完成相关
生成标题: 在智慧柜员机上激活信用卡的方法如下：<br/> 1、登录个人网银，通过信用卡信用卡管理信用卡管理信用卡激活菜单，点击激活按钮激活信用卡。<br/> 2、登

Epoch 6/40
2033/2033 [==============================] - 1685s 829ms/step - loss: 2.8114
生成标题: 我需要完成资产证明的任务。
生成标题: 在智慧柜员机上激活信用卡的方法如下：<br/> 1、登录个人网银

KeyboardInterrupt: ignored

In [0]:
s = "你好我想问问怎样取钱"

In [0]:
s1 = "你好,请问怎样存款"

In [0]:
s2 = "你好，我想问问银行卡的余额还剩多少"

In [0]:
s3 = "信用卡超额还款"

In [0]:
s4 = "信用卡办理条件"

In [0]:
s5 = "单身狗怎么过520节日呢"

In [0]:
autoanswer2 = AutoAnswer(start_id=None, end_id=tokenizer._token_end_id, maxlen=96)

In [29]:
print(u'生成回答:', autoanswer2.generate(s))

生成回答: 您好，您可以到智慧柜员机办理取款业务，取款时请您签字，取款申请书，取款时请您出示身份证件。


In [30]:
print(u'生成回答:', autoanswer2.generate(s1))

生成回答: 您好, 您可以到自助存款机上存款。


In [31]:
print(u'生成回答:',autoanswer2.generate(s2))

生成回答: 您好，目前我行暂时还不支持余额查询。


In [37]:
print(u'生成回答:', autoanswer2.generate(s3))

生成回答: 您好，如果您的信用卡超过信用额度，您可以通过我行网点、网点柜面、自助终端、网点柜员机等渠道进行还款。


In [40]:
print(u'生成回答:', autoanswer2.generate(s4))

生成回答: 信用卡申办条件是：具有完全民事行为能力、年满18周岁且未满70周岁、具有稳定的职业和收入、信用状况良好、有按时还本付息能力的自然人，可申请龙卡信用卡个人卡主卡。


In [47]:
print(u'生成回答:', autoanswer2.generate(s5))

生成回答: 祝您每天都快乐，每天都快乐！
